In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,11123
2,Huelva,Confirmados PDIA 14 días,837
3,Huelva,Tasa PDIA 14 días,"163,87889799762308"
4,Huelva,Confirmados PDIA 7 días,323
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,2
629,Municipio de Huelva sin especificar,Total Confirmados,126
630,Municipio de Huelva sin especificar,Curados,107


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  11123.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3145.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1451 personas en los últimos 7 días 

Un positivo PCR cada 591 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,11123.0,323.0,837.0,510743.0,63.241200,163.878898,44.0
Huelva-Costa,6286.0,215.0,545.0,288115.0,74.622980,189.160578,32.0
Huelva (capital),3145.0,99.0,243.0,143663.0,68.911272,169.145848,15.0
Condado-Campiña,3426.0,103.0,265.0,155057.0,66.427185,170.904893,13.0
Cartaya,423.0,32.0,74.0,19974.0,160.208271,370.481626,9.0
Lepe,970.0,23.0,67.0,27431.0,83.846743,244.249207,5.0
Palma del Condado (La),545.0,23.0,63.0,10761.0,213.734783,585.447449,5.0
Almonte,447.0,9.0,13.0,24191.0,37.203919,53.738994,3.0
Moguer,382.0,14.0,44.0,22088.0,63.382832,199.203187,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,104.0,8.0,21.0,3371.0,237.318303,622.960546,0.0
Palma del Condado (La),545.0,23.0,63.0,10761.0,213.734783,585.447449,5.0
Bonares,66.0,15.0,23.0,6058.0,247.606471,379.663255,1.0
Cartaya,423.0,32.0,74.0,19974.0,160.208271,370.481626,9.0
Isla Cristina,450.0,31.0,73.0,21264.0,145.786305,343.303236,2.0
Palos de la Frontera,251.0,15.0,35.0,11289.0,132.872708,310.036319,0.0
Escacena del Campo,97.0,1.0,6.0,2284.0,43.782837,262.697023,0.0
Lepe,970.0,23.0,67.0,27431.0,83.846743,244.249207,5.0
Aljaraque,439.0,14.0,47.0,21260.0,65.851364,221.072437,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Sierra de Huelva-Andévalo Central,1286.0,3.0,23.0,67571.0,4.439774,34.038271,0.0,0.130435
San Juan del Puerto,154.0,2.0,15.0,9300.0,21.505376,161.290323,-1.0,0.133333
Bollullos Par del Condado,250.0,2.0,15.0,14272.0,14.013453,105.100897,1.0,0.133333
Escacena del Campo,97.0,1.0,6.0,2284.0,43.782837,262.697023,0.0,0.166667
Aljaraque,439.0,14.0,47.0,21260.0,65.851364,221.072437,1.0,0.297872
Moguer,382.0,14.0,44.0,22088.0,63.382832,199.203187,3.0,0.318182
Gibraleón,350.0,1.0,3.0,12607.0,7.932101,23.796304,0.0,0.333333
Lepe,970.0,23.0,67.0,27431.0,83.846743,244.249207,5.0,0.343284
Palma del Condado (La),545.0,23.0,63.0,10761.0,213.734783,585.447449,5.0,0.365079
